In [1]:
import pickle
import numpy as np
import pandas as pd
from langchain_core.tools import tool
from keras.saving import load_model
from pydantic import BaseModel, Field, validator

class DataRow(BaseModel):
    age: int                                 = Field(description="Your age in years")
    gender_Male: bool                        = Field(description="1 if you're a male, 0 otherwise")
    gender_Female: bool                      = Field(description="1 if you're a female, 0 otherwise")
    gender_Other: bool                       = Field(description="1 if you're not male nor female, 0 otherwise")
    hypertension: int                        = Field(description="Do you have hypertension? (0, 1)")
    heart_disease: int                       = Field(description="Do you have any heart disease? (0, 1)")
    avg_glucose_level: float                 = Field(description="Your average glucose level in mg/dL")
    bmi: int                                 = Field(description="Your Body Mass Index (BMI)")
    ever_married_No: bool                    = Field(description="0 if you've ever been married")
    ever_married_Yes: bool                   = Field(description="1 if you've ever been married")
    work_type_Govt_job: bool                 = Field(description="1 if you have a govt_job, 0 otherwise")
    work_type_Never_worked: bool             = Field(description="1 if you never worked, 0 otherwise")
    work_type_Private: bool                  = Field(description="1 if you have a private job, 0 otherwise")
    work_type_Self_employed: bool            = Field(description="1 if you have a self employed job, 0 otherwise")
    work_type_children: bool                 = Field(description="1 if you are a child, 0 otherwise")
    Residence_type_Rural: bool               = Field(description="1 if you live in a rural residence, 0 otherwise")
    Residence_type_Urban: bool               = Field(description="1 if you live in a urban residence, 0 otherwise")
    smoking_status_Unknown: bool             = Field(description="1 if your smoking status is unknown, 0 otherwise")
    smoking_status_formerly_smoked: bool     = Field(description="1 if you don't smoke anymore, 0 otherwise")
    smoking_status_never_smoked: bool        = Field(description="1 if you never smoked, 0 otherwise")
    smoking_status_smokes: bool              = Field(description="1 if you smoke, 0 otherwise")

@tool
def get_weather(location: str):
    """Returns the current weather for the given location."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."

@tool("get_stroke_risk", args_schema=DataRow, return_direct=True)
def get_stroke_risk(
    age: int,
    gender_Male: bool,
    gender_Female: bool,
    gender_Other: bool,
    hypertension: int,
    heart_disease: int,
    avg_glucose_level: float,
    bmi: int,
    ever_married_No: bool,
    ever_married_Yes: bool,
    work_type_Govt_job: bool,
    work_type_Never_worked: bool,
    work_type_Private: bool,
    work_type_Self_employed: bool,
    work_type_children: bool,
    Residence_type_Rural: bool,
    Residence_type_Urban: bool,
    smoking_status_Unknown: bool,
    smoking_status_formerly_smoked: bool,
    smoking_status_never_smoked: bool,
    smoking_status_smokes: bool,
):
    """Loads a ML model and makes a prediction"""
    # tensor = np.array([gender, age, hypertension, heart_disease, marital_status, work_type, residence_type, avg_glucose_level, bmi, smoking_status]).reshape(-1, 1)
    model = load_model("model/model.keras")
    print(age)
    tensor = np.array([
        age,
        gender_Male,
        gender_Female,
        gender_Other,
        hypertension,
        heart_disease,
        avg_glucose_level,
        bmi,
        ever_married_No,
        ever_married_Yes,
        work_type_Govt_job,
        work_type_Never_worked,
        work_type_Private,
        work_type_Self_employed,
        work_type_children,
        Residence_type_Rural,
        Residence_type_Urban,
        smoking_status_Unknown,
        smoking_status_formerly_smoked,
        smoking_status_never_smoked,
        smoking_status_smokes
    ])
    tensor = np.array([[54, 1, 0, 0, 1, 1, 135.4, 28, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])
    return (model.predict(tensor) > 0.4).astype(int)


In [2]:
get_stroke_risk.invoke({
    "age": 54,
    "gender_Male": 1,
    "gender_Female": 0,
    "gender_Other": 0,
    "hypertension": 1,
    "heart_disease": 1,
    "avg_glucose_level": 135.4,
    "bmi": 28,
    "ever_married_No": 0,
    "ever_married_Yes": 1,
    "work_type_Govt_job": 1,
    "work_type_Never_worked": 0,
    "work_type_Private": 0,
    "work_type_Self_employed": 0,
    "work_type_children": 0,
    "Residence_type_Rural": 0,
    "Residence_type_Urban": 1,
    "smoking_status_Unknown": 0,
    "smoking_status_formerly_smoked": 0,
    "smoking_status_never_smoked": 1,
    "smoking_status_smokes": 0
})

54
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


array([[0]])

In [3]:

from langgraph.prebuilt import ToolNode

tools = [get_weather, get_stroke_risk]
tool_node = ToolNode(tools)

In [4]:
from langchain_openai import ChatOpenAI  # pip install langchain-openai

llm = ChatOpenAI(
    openai_api_key="YOUR OPENAI API KEY HERE", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
).bind_tools(tools)

In [5]:
from typing import Literal

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver


def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "what's the weather in sf?")]}, stream_mode="values", config={"configurable": {"thread_id": "1"}}
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

what's the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_UVF0rOfLpfQdrcQNoChhOZBo)
 Call ID: call_UVF0rOfLpfQdrcQNoChhOZBo
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: get_weather

It's 60 degrees and foggy.
================================== Ai Message ==================================

The weather in San Francisco is currently 60 degrees Fahrenheit and foggy.


In [7]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "Do you have any special tool?")]}, stream_mode="values", config={"configurable": {"thread_id": "1"}}
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Do you have any special tool?
================================== Ai Message ==================================

Yes, I have access to a variety of tools that can provide information, make predictions, perform calculations, and more. If you have a specific task or question in mind, feel free to let me know, and I can utilize the appropriate tool to assist you.


In [8]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "i want to know if i am at risk stroke?")]}, stream_mode="values", config={"configurable": {"thread_id": "1"}}
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

i want to know if i am at risk stroke?
================================== Ai Message ==================================

To determine your risk of stroke, I will need some information. Could you please provide me with the following details:

1. Your age in years.
2. Your gender (Male, Female, Other).
3. Do you have hypertension? (Yes or No)
4. Do you have any heart disease? (Yes or No)
5. Your average glucose level in mg/dL.
6. Your Body Mass Index (BMI).
7. Have you ever been married? (Yes or No)
8. Your work type (Govt_job, Never_worked, Private, Self_employed, Children).
9. Your residence type (Rural or Urban).
10. Your smoking status (Unknown, Formerly smoked, Never smoked, Smokes).

Please provide this information so that I can assess your risk of stroke.


In [9]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "how can i pass the information to you?")]}, stream_mode="values", config={"configurable": {"thread_id": "1"}}
):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

how can i pass the information to you?
================================== Ai Message ==================================

You can provide the necessary information for assessing your risk of stroke by answering the questions I listed in the previous message. Simply respond with the relevant details, and I will use that information to determine your risk of stroke. Let's start with the first question:

1. What is your age in years?


In [10]:
# example with a single tool call

description = """
Alex Rivera is a seasoned professional in his mid-50s, known for his resilience and dedication to public service. Having spent years in government roles, he is well-acquainted with the intricacies of administrative work. He enjoys life in a lively urban setting, where the pace and culture match his dynamic lifestyle.

Alex's health journey includes managing certain conditions, which he handles with a proactive approach, maintaining a balanced lifestyle. Family is central to his life, and with a long-standing marriage, he finds great joy in spending time with his loved ones, especially sharing stories and wisdom with the younger generation. A non-smoker with a disciplined routine, Alex is a firm believer in nurturing well-being for himself and those around him.
"""


for chunk in app.stream(
    {"messages": [("human", description)]}, stream_mode="values", config={"configurable": {"thread_id": "1"}}
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================


Alex Rivera is a seasoned professional in his mid-50s, known for his resilience and dedication to public service. Having spent years in government roles, he is well-acquainted with the intricacies of administrative work. He enjoys life in a lively urban setting, where the pace and culture match his dynamic lifestyle.

Alex's health journey includes managing certain conditions, which he handles with a proactive approach, maintaining a balanced lifestyle. Family is central to his life, and with a long-standing marriage, he finds great joy in spending time with his loved ones, especially sharing stories and wisdom with the younger generation. A non-smoker with a disciplined routine, Alex is a firm believer in nurturing well-being for himself and those around him.

================================== Ai Message ==================================
Tool Calls:
  get_stroke_risk (call_PVjUJ3eDhj31edXqEi2Q8Gum)
 C

In [14]:
app.invoke(
    {"messages": ["what's the name of the guy?"]}, 
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="what's the weather in sf?", additional_kwargs={}, response_metadata={}, id='b542d7f6-0d75-483f-85ee-56964f38e48c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UVF0rOfLpfQdrcQNoChhOZBo', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 493, 'total_tokens': 508, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8bc90eab-ce1d-4ef0-8d80-bef754f9b91d-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_UVF0rOfLpfQdrcQNoChhOZBo', 'type': 'tool_call'}], usage_metadata={'in